# Part 4 - Word embedding with Word2Vec and GloVe

Author: Laura Mercy

Date: 2022-12-12

Contact: laura.mercym@gmail.com

In Part 3, we vectorized text data using the Bag-of-Words and the TF-IDF technique. As we've seen, they are high dimensional (the first sparse matrix on description was almost 100,000 dimensions!) and therefore computationally expensive.

Word embedddings on the other hand, are produced by neural networks and usually between 100 and 600 dimensions, which makes them a lot faster to run machine learning models on. 

Unlike Bag-of-Words which only counts the number of tokens, word embeddings allow to look at words based on their meaning. Words with similar meanings will have similar vectors and will be neighbors in the embedding space.

We won't be looking at similarity too much in this project, so our main interest for word embedding is their small dimensions for running faster models.

In this notebook, we will transform again title and description, but using 2 word embedding methods, Word2Vec and GloVe on pre-trained models with Wikipedia vocabulary. We will then run the same models that we ran in part 2, and compare the performance.

**Important! Before starting, you will need to do the following:**
- Set the kernel of the Jupyter notebook to the 'deeplearning' environment that was created in class.
- Download on your local drive the LexVex file that was included in the submission package or get it [here](https://drive.google.com/file/d/1goH3DeSJkpUACmXzezc9GoagwHEtsJ6Q/view?usp=share_link)
- Download on your local the GloVe file that was included in the submission package or get it [here](https://drive.google.com/file/d/1hZNsHzCQR5rGHJDO8nKSHTLBlCDNLEpM/view?usp=share_link)

We start by importing all the librairies we will need for our work, and the same clean dataset that was produced in Part 2.

In [1]:
#import the basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split


In [2]:
#import basic libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#to display the full column
pd.set_option('display.max_columns', None)

#to scale the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

#To split the dataset and do a GridSearch
from sklearn.model_selection import train_test_split, GridSearchCV

#To run machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#To reduce dimensionality
from sklearn.decomposition import PCA

#To score the model
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

# To set up a temporary directory for caching pipeline results
from tempfile import mkdtemp

# To build a pipeline
from sklearn.pipeline import Pipeline

# To get rid of warnings that appear if sklearn hasn't been updated
import warnings
warnings.filterwarnings('ignore')

#To do cross validation
from sklearn.model_selection import cross_val_score

In [3]:
#importing our clean data
books_df=pd.read_csv('data/books_df_clean.csv')

In [4]:
#Checking the data
books_df.head()

,Unnamed: 0,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,is_popular
0,0,The Hunger Games,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,374,3,1,12,147,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,2008,1
1,1,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,870,8,1,24,136,0,1,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,2004,1
2,2,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,324,4,1,2,137,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,2006,1
3,3,Pride and Prejudice,Alternate cover edition of ISBN 9780679783268S...,279,3,0,14,100,1,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,2000,1
4,4,Twilight,About three things I was absolutely positive.\...,501,1,1,14,55,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,2009,0


In [5]:
#Dropping Unnamed column
books_df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
books_df.head(1)

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,is_popular
0,The Hunger Games,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,374,3,1,12,147,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,2008,1


Finally, we assign the same X and y variables as in the previous notebooks, and split the dataset between train and test set with the same arguments as before.

In [7]:
#Assigning X and y
X=books_df.drop('is_popular',axis=1)
y=books_df['is_popular']

In [8]:
# Split the data to train and test set
X_train, X_test, y_train, y_test= train_test_split (X, y, test_size=0.2, stratify=y, random_state=1)

We are now ready to start.

# 1. Word2Vec

As mentioned in the introduction, we will not train a word embedding model ourselves, we will use pretrained vectors: LexVec, that was trained on Wikipedia vocabulary. We will use the gensim library in Python to interact with vectors. 

In [9]:
#import the gensim library to work with vectors
import gensim

In [10]:
#importing a pre-trained vector LexVex
model = gensim.models.KeyedVectors.load_word2vec_format(
    'data/lexvec.enwiki+newscrawl.300d.W.pos.vectors', binary=False
)

In [11]:
#importing library to calculate cosine distance
from scipy.spatial.distance import cosine as cosine_distance

In [12]:
#importing a function to embed a text
from gensim.utils import simple_preprocess

Now, we have the pre-trained model but we need to associate the corresponding vector to each word in our corpus. In fact, what we really need is a vector representation of the text we are embedding, the title and the description. If the description has 100 words, we don't want to have 100 vectors, we want one vector only for this one description.

In order to get that, we need to create a function that will:
1. Split each document into tokens
2. Look at each token and add the vector to a numpy array IF the token is in the LexVex model
3. Average all the vectors of one document into one final vector

We will use the function below that was provided in the Representation Learning lecture:

In [13]:
#Create a function to vectorize the description

def sentence2vec(text):
    """
    Embed a text by averaging the word vectors of the tokenized text. Out-of-vocabulary words are replaced by the zero-vector.
    -----
    
    Input: text (string)
    Output: embedding vector (np.array)
    """
    tokenized = simple_preprocess(text)
    
    word_embeddings = [np.zeros(300)]
    for word in tokenized:
        # if the word is in the model then embed
        if word in model:
            vector = model[word]
        # add zeros for out-of-vocab words
        else:
            vector = np.zeros(300)
            
        word_embeddings.append(vector)
    
    # average the word vectors
    sentence_embedding = np.stack(word_embeddings).mean(axis=0)
    
    return sentence_embedding

## 1.1 Converting Description

Now that the function is ready, we want convert the train and test set into vectors. There are no risk of data leakage between the train and test set here, since we are using a pre-trained model. We are simply using the model to vectorize the text. But for good practice, we will follow the same steps are before: split the dataset, convert description and title, and merge with the rest of the features.

### 1.1.1 Train set

We will apply the sentence2vec function to teh description in X_train, put the results into a dataframe, then merge the vectorized words with the rest of the features.

In [14]:
#Applying sentence2vec to X_train description
X_train_emb_desc = X_train['description'].apply(sentence2vec)

X_train_emb_desc.head()

12988    [0.03440669484740291, 0.01096203031753529, 0.0...
4460     [0.026139167081858324, 0.02973891910310281, 0....
19577    [0.02922946210944643, 0.023471153317353666, 0....
35307    [0.007315265179293382, 0.03725596947942347, 0....
12699    [-0.012442224228155168, 0.011428701282035955, ...
Name: description, dtype: object

In [15]:
#Putting the results into a numpy array
X_train_emb_desc_np = np.array(X_train_emb_desc.tolist())

In [16]:
#Creating a dataframe
X_train_emb_desc_df=pd.DataFrame(X_train_emb_desc_np, columns=range(0,300,1),index=X_train_emb_desc.index)

#Adding _d in front of the column name
X_train_emb_desc_df=X_train_emb_desc_df.add_prefix("d_")
                            
X_train_emb_desc_df

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
12988,0.034407,0.010962,0.040430,-0.048449,-0.059889,0.047578,-0.084650,-0.048583,0.022233,0.014557,0.071009,0.044433,-0.031707,0.034787,0.009703,-0.014388,-0.068466,0.031361,-0.006960,-0.009206,-0.058156,0.010157,0.003321,-0.009987,-0.050872,-0.005597,0.063596,0.119994,0.071549,0.003738,-0.069750,-0.030606,0.008556,0.003742,-0.004082,0.018875,0.029516,-0.051441,-0.008393,0.016633,0.064862,-0.023847,-0.030752,-0.044723,-0.020524,-0.011085,0.030305,-0.003017,-0.036495,0.023091,-0.006456,0.003618,-0.000456,-0.010354,-0.036037,0.024405,0.029477,0.007259,-0.106552,0.004044,0.057341,-0.039839,-0.043380,0.004250,0.046244,0.078353,0.045713,-0.044337,-0.056833,-0.000400,0.033133,0.060246,-0.034894,0.053449,-0.014983,0.011639,-0.015986,-0.066267,0.048070,0.061700,0.061581,0.002411,-0.018499,-0.006626,0.003228,-0.032577,-0.036531,0.012183,-0.017123,-0.044578,0.013769,0.001692,-0.010871,0.036700,0.022954,-0.018483,-0.041645,0.061167,-0.012418,0.027908,0.008211,0.039775,-0.004859,0.007478,0.018217,-0.010155,-0.112120,-0.055916,-0.015222,0.000347,-0.079957,0.028167,0.058104,0.000184,0.132203,-0.028715,-0.002798,0.012156,-0.061197,-0.018045,0.023504,0.045004,-0.044241,0.027018,0.029903,0.013213,0.011777,0.032742,-0.015491,0.053792,0.044620,-0.077073,0.021937,-0.051415,-0.050496,0.049257,-0.035554,-0.107342,0.049315,0.012875,0.001153,-0.044610,-0.002263,0.009140,0.013224,0.005978,0.032772,0.035288,-0.059793,0.015772,-0.054485,0.063953,-0.010693,-0.039396,-0.009002,0.012244,-0.061761,0.001490,-0.015568,-0.025654,0.040624,0.002916,0.064876,0.014240,-0.038980,0.025407,0.048449,-0.006828,-0.005806,0.015071,-0.018255,0.062245,0.090538,0.020385,-0.002431,-0.002402,-0.039129,-0.064093,0.007788,0.017144,-0.047388,0.078424,-0.019660,0.000619,0.042963,0.012218,-0.095672,0.071500,0.015620,-0.012066,-0.025510,0.018418,0.003947,-0.061543,0.058126,0.018748,-0.020571,0.011560,-0.017313,-0.051188,0.047259,0.001366,0.052391,-0.001801,-0.014354,-0.014530,-0.018460,-0.001340,-0.074451,0.066634,0.019205,-0.011456,0.040121,0.045823,0.029803,-0.053912,0.003293,-0.051404,0.049369,-0.007184,-0.008154,-0.062575,-0.110244,0.061311,-0.044454,-0.030056,0.005181,-0.059516,0.037542,-0.061658,0.011344,0.017545,-0.051460,0.018202,0.077499,0.009770,0.032262,0.045612,-0.034430,0.048558,0.019315,-0.047411,0.000875,0.0

In [17]:
#checking that the number of rows is the same
X_train.shape[0]-X_train_emb_desc_df.shape[0]
#Expect 0

0

Since the index of this dataframe is the same as the Train index, we can directly merge the 2 dataframes without having to reset the Train index as we did in Part 3.

In [18]:
#Checking X_Train
X_train.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year
12988,The Radium Girls: The Dark Story of America's ...,\n The incredible true story of the women who...,479,10,0,1,231,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2017
4460,The Dark Web Murders,I AM MEMEIN. I AM EMOTIONALLY DETACHED FROM MY...,416,4,1,2,238,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019
19577,I Sold Myself To The Devil For Vinyls... Pitif...,"Lexi Grayson is just a normal teenager, as nor...",421,11,1,1,38,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,2014
35307,The Devil Knows You're Dead,"Scudder is back, tracking a killer through the...",338,5,1,14,99,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1999
12699,Chariots of The Gods,Erich von Daniken's Chariots of the Gods is a ...,220,4,0,2,178,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1999


In [19]:
#Checking the new dataframe
X_train_emb_desc_df.head()

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
12988,0.034407,0.010962,0.040430,-0.048449,-0.059889,0.047578,-0.084650,-0.048583,0.022233,0.014557,0.071009,0.044433,-0.031707,0.034787,0.009703,-0.014388,-0.068466,0.031361,-0.006960,-0.009206,-0.058156,0.010157,0.003321,-0.009987,-0.050872,-0.005597,0.063596,0.119994,0.071549,0.003738,-0.069750,-0.030606,0.008556,0.003742,-0.004082,0.018875,0.029516,-0.051441,-0.008393,0.016633,0.064862,-0.023847,-0.030752,-0.044723,-0.020524,-0.011085,0.030305,-0.003017,-0.036495,0.023091,-0.006456,0.003618,-0.000456,-0.010354,-0.036037,0.024405,0.029477,0.007259,-0.106552,0.004044,0.057341,-0.039839,-0.043380,0.004250,0.046244,0.078353,0.045713,-0.044337,-0.056833,-0.000400,0.033133,0.060246,-0.034894,0.053449,-0.014983,0.011639,-0.015986,-0.066267,0.048070,0.061700,0.061581,0.002411,-0.018499,-0.006626,0.003228,-0.032577,-0.036531,0.012183,-0.017123,-0.044578,0.013769,0.001692,-0.010871,0.036700,0.022954,-0.018483,-0.041645,0.061167,-0.012418,0.027908,0.008211,0.039775,-0.004859,0.007478,0.018217,-0.010155,-0.112120,-0.055916,-0.015222,0.000347,-0.079957,0.028167,0.058104,0.000184,0.132203,-0.028715,-0.002798,0.012156,-0.061197,-0.018045,0.023504,0.045004,-0.044241,0.027018,0.029903,0.013213,0.011777,0.032742,-0.015491,0.053792,0.044620,-0.077073,0.021937,-0.051415,-0.050496,0.049257,-0.035554,-0.107342,0.049315,0.012875,0.001153,-0.044610,-0.002263,0.009140,0.013224,0.005978,0.032772,0.035288,-0.059793,0.015772,-0.054485,0.063953,-0.010693,-0.039396,-0.009002,0.012244,-0.061761,0.001490,-0.015568,-0.025654,0.040624,0.002916,0.064876,0.014240,-0.038980,0.025407,0.048449,-0.006828,-0.005806,0.015071,-0.018255,0.062245,0.090538,0.020385,-0.002431,-0.002402,-0.039129,-0.064093,0.007788,0.017144,-0.047388,0.078424,-0.019660,0.000619,0.042963,0.012218,-0.095672,0.071500,0.015620,-0.012066,-0.025510,0.018418,0.003947,-0.061543,0.058126,0.018748,-0.020571,0.011560,-0.017313,-0.051188,0.047259,0.001366,0.052391,-0.001801,-0.014354,-0.014530,-0.018460,-0.001340,-0.074451,0.066634,0.019205,-0.011456,0.040121,0.045823,0.029803,-0.053912,0.003293,-0.051404,0.049369,-0.007184,-0.008154,-0.062575,-0.110244,0.061311,-0.044454,-0.030056,0.005181,-0.059516,0.037542,-0.061658,0.011344,0.017545,-0.051460,0.018202,0.077499,0.009770,0.032262,0.045612,-0.034430,0.048558,0.019315,-0.047411,0.000875,0.0

One of the main use case for word embedding is to check the similarities between vectors. We can see how close the description of the first row is to the description in the second row and 3rd row by looking at their cosine distance:

In [20]:
#Distance between the description of row 1 and row 2
cosine_distance(X_train_emb_desc_df.iloc[0], X_train_emb_desc_df.iloc[1])

0.16393860984543285

In [21]:
#Distance between the description of row 1 and row 3
cosine_distance(X_train_emb_desc_df.iloc[0], X_train_emb_desc_df.iloc[2])

0.31957507214421976

We see that the first and second row's description are more similar thant the first and third row, as their cosine distance is smaller, which means their are closer together in the embedding space. But looking at similar books in beyond the scope of this project. Let's keep going.

In [22]:
#Merging the 2
X_train_w2v=pd.merge(X_train,X_train_emb_desc_df,left_index=True,right_index=True)

In [23]:
#Checking the result
X_train_w2v.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
12988,The Radium Girls: The Dark Story of America's ...,\n The incredible true story of the women who...,479,10,0,1,231,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2017,0.034407,0.010962,0.040430,-0.048449,-0.059889,0.047578,-0.084650,-0.048583,0.022233,0.014557,0.071009,0.044433,-0.031707,0.034787,0.009703,-0.014388,-0.068466,0.031361,-0.006960,-0.009206,-0.058156,0.010157,0.003321,-0.009987,-0.050872,-0.005597,0.063596,0.119994,0.071549,0.003738,-0.069750,-0.030606,0.008556,0.003742,-0.004082,0.018875,0.029516,-0.051441,-0.008393,0.016633,0.064862,-0.023847,-0.030752,-0.044723,-0.020524,-0.011085,0.030305,-0.003017,-0.036495,0.023091,-0.006456,0.003618,-0.000456,-0.010354,-0.036037,0.024405,0.029477,0.007259,-0.106552,0.004044,0.057341,-0.039839,-0.043380,0.004250,0.046244,0.078353,0.045713,-0.044337,-0.056833,-0.000400,0.033133,0.060246,-0.034894,0.053449,-0.014983,0.011639,-0.015986,-0.066267,0.048070,0.061700,0.061581,0.002411,-0.018499,-0.006626,0.003228,-0.032577,-0.036531,0.012183,-0.017123,-0.044578,0.013769,0.001692,-0.010871,0.036700,0.022954,-0.018483,-0.041645,0.061167,-0.012418,0.027908,0.008211,0.039775,-0.004859,0.007478,0.018217,-0.010155,-0.112120,-0.055916,-0.015222,0.000347,-0.079957,0.028167,0.058104,0.000184,0.132203,-0.028715,-0.002798,0.012156,-0.061197,-0.018045,0.023504,0.045004,-0.044241,0.027018,0.029903,0.013213,0.011777,0.032742,-0.015491,0.053792,0.044620,-0.077073,0.021937,-0.051415,-0.050496,0.049257,-0.035554,-0.107342,0.049315,0.012875,0.001153,-0.044610,-0.002263,0.009140,0.013224,0.005978,0.032772,0.035288,-0.059793,0.015772,-0.054485,0.063953,-0.010693,-0.039396,-0.009002,0.012244,-0.061761,0.001490,-0.015568,-0.

The merged dataframe looks good. 

One important things to note here, is that the vectorization of the text did not give us tokens as it did with Bag-of-Words, it gaves us vectors with 300 dimensions. Each column correspond to a dimension of the vector of the whole description for this one row. Therefore we won't be able to interpret which words have a higher predictive power, however the vectorization can still help us to run predictions on the popularity of a book. 

### 1.1.2 Test set

We'll repeat the same steps above with the test set.

In [24]:
#Applying sentence2vec to X_train title
X_test_emb_desc = X_test['description'].apply(sentence2vec)

#Putting the results into a numpy array
X_test_emb_desc_np = np.array(X_test_emb_desc.tolist())

#Creating a dataframe
X_test_emb_desc_df=pd.DataFrame(X_test_emb_desc_np, columns=range(0,300,1),index=X_test_emb_desc.index)

#Adding _d in front of the column name
X_test_emb_desc_df=X_test_emb_desc_df.add_prefix("d_")
                            
#Merging the 2
X_test_w2v=pd.merge(X_test,X_test_emb_desc_df,left_index=True,right_index=True)

#checking the new DF
X_test_w2v.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
35899,Legends over Generations,"Since the beginning of human settlement, a lot...",228,3,0,1,188,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2018,0.016739,0.024467,0.064448,-0.093333,-0.001788,0.051264,-0.100052,-0.067099,0.009811,0.015671,0.088806,0.036483,-0.010852,0.025935,0.009727,-0.039675,-0.075722,-0.001945,-0.003147,0.012218,-0.040296,-0.018493,0.035588,0.003640,-0.066411,-0.026287,0.057094,0.134299,0.028739,0.008663,-0.067807,-0.031211,0.012624,0.002968,-0.046042,0.034360,0.031306,-0.053635,-0.043331,-0.022810,0.083525,0.004502,-0.027294,-0.060101,-0.024506,-0.041472,-0.000355,-0.001263,-0.045007,-0.025978,0.003585,-0.014156,0.022842,-0.052204,-0.008659,0.005463,0.037297,0.027925,-0.082994,-0.007898,0.054826,-0.029306,-0.031780,0.010119,0.061192,0.065588,0.046314,-0.103930,-0.041344,0.041222,0.002771,0.047826,-0.013336,0.069930,-0.004617,-0.001912,0.009526,-0.067580,0.040730,0.077719,0.032842,0.023725,0.007861,0.004889,-0.021075,-0.023601,-0.056579,-0.014842,-0.032221,-0.037949,0.012670,-0.007385,0.040513,0.014416,0.009946,-0.003087,-0.034999,0.056449,-0.074759,0.061882,-0.001484,0.052071,0.015206,0.003346,0.025719,-0.046630,-0.094215,-0.059869,-0.000595,-0.013194,-0.070122,0.045488,0.059736,0.008544,0.142893,-0.031151,-0.010937,-0.010458,-0.056980,-0.038334,0.033925,0.076599,-0.058297,0.021382,0.052126,0.036713,0.007371,0.040062,-0.019412,0.063582,0.057064,-0.030138,0.029002,-0.031066,-0.010459,0.036309,-0.057199,-0.114052,0.023933,0.023416,0.001289,-0.067542,-0.019362,0.004060,-0.005235,0.013702,0.032791,0.031591,-0.054310,0.023779,-0.078012,0.027572,0.013581,-0.034142,-0.000737,0.014244,-0.064259,0.042991,-0.030341,0.005174,0.044307,-0.00

## 1.2 Converting Title

We repeat the same steps for the title text, on the train and test dataset.

### 1.2.1 Train set

In [25]:
#Applying sentence2vec to X_train title
X_train_emb_title = X_train['title'].apply(sentence2vec)

#Putting the results into a numpy array
X_train_emb_title_np = np.array(X_train_emb_title.tolist())

#Creating a dataframe
X_train_emb_title_df=pd.DataFrame(X_train_emb_title_np, columns=range(0,300,1),index=X_train_emb_title.index)

#Adding _t in front of the column name
X_train_emb_title_df=X_train_emb_title_df.add_prefix("t_")
                            
#Merging with the dataframe created earlier with the description vectors
X_train_w2v=pd.merge(X_train_w2v,X_train_emb_title_df,left_index=True,right_index=True)

#checking the new DF
X_train_w2v.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59,t_60,t_61,t_62,t_63,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,t_100,t_101,t_102,t_103,t_104,t_105,t_106,t_107,t_108,t_109,t_110,t_111,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121,t_122,t_123,t_124,t_125,t_126,t_127,t_128,t_129,t_130,t_131,t_132,t_133,t_134,t_135,t_136,t_137,t_138,t_139,t_140,t_141,t_142,t_143,t_144,t_145,t_146,t_147,t_148,t_149,t_150,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160,t_161,t_162,t_163,t_164,t_165,t_166,t_167,t_168,t_169,t_170,t_171,t_172,t_173,t_174,t_175,t_176,t_177,t_178,t_179,t_180,t_181,t_182,t_183,t_184,t_185,t_186,t_187,t_188,t_189,t_190,t_191,t_192,t_193,t_194,t_195,t_196,t_197,t_198,t_199,t_200,t_201,t_202,t_203,t_204,t_205,t_206,t_207,t_208,t_209,t_210,t_211,t_212,t_213,t_214,t_215,t_216,t_217,t_218,t_219,t_220,t_221,t_222,t_223,t_224,t_225,t_226,t_227,t_228,t_229,t_230,t_231,t_232,t_233,t_234,t_235,t_236,t_237,t_238,t_239,t_240,t_241,t_242,t_243,t_244,t_245,t_246,t_247,t_248,t_249,t_250,t_251,t_252,t_253,t_254,t_255,t_256,t_257,t_258,t_259,t_260,t_261,t_262,t_263,t_264,t_265,t_266,t_267,t_268,t_269,t_270,t_271,t_272,t_273,t_274,t_275,t_276,t_277,t_278,t_279,t_280,t_281,t_282,t_283,t_284,t_285,t_286,t_287,t_288,t_289,t_290,t_291,t_292,t_293,t_294,t_295,t_296,t_297,t_298,t_299
12988,Th

### 1.2.2 Test set

In [26]:
#Applying sentence2vec to X_test title
X_test_emb_title = X_test['title'].apply(sentence2vec)

#Putting the results into a numpy array
X_test_emb_title_np = np.array(X_test_emb_title.tolist())

#Creating a dataframe
X_test_emb_title_df=pd.DataFrame(X_test_emb_title_np, columns=range(0,300,1),index=X_test_emb_title.index)

#Adding _t in front of the column name
X_test_emb_title_df=X_test_emb_title_df.add_prefix("t_")
                            
#Merging with the dataframe created earlier with the description vectors
X_test_w2v=pd.merge(X_test_w2v,X_test_emb_title_df,left_index=True,right_index=True)

#checking the new DF
X_test_w2v.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59,t_60,t_61,t_62,t_63,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,t_100,t_101,t_102,t_103,t_104,t_105,t_106,t_107,t_108,t_109,t_110,t_111,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121,t_122,t_123,t_124,t_125,t_126,t_127,t_128,t_129,t_130,t_131,t_132,t_133,t_134,t_135,t_136,t_137,t_138,t_139,t_140,t_141,t_142,t_143,t_144,t_145,t_146,t_147,t_148,t_149,t_150,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160,t_161,t_162,t_163,t_164,t_165,t_166,t_167,t_168,t_169,t_170,t_171,t_172,t_173,t_174,t_175,t_176,t_177,t_178,t_179,t_180,t_181,t_182,t_183,t_184,t_185,t_186,t_187,t_188,t_189,t_190,t_191,t_192,t_193,t_194,t_195,t_196,t_197,t_198,t_199,t_200,t_201,t_202,t_203,t_204,t_205,t_206,t_207,t_208,t_209,t_210,t_211,t_212,t_213,t_214,t_215,t_216,t_217,t_218,t_219,t_220,t_221,t_222,t_223,t_224,t_225,t_226,t_227,t_228,t_229,t_230,t_231,t_232,t_233,t_234,t_235,t_236,t_237,t_238,t_239,t_240,t_241,t_242,t_243,t_244,t_245,t_246,t_247,t_248,t_249,t_250,t_251,t_252,t_253,t_254,t_255,t_256,t_257,t_258,t_259,t_260,t_261,t_262,t_263,t_264,t_265,t_266,t_267,t_268,t_269,t_270,t_271,t_272,t_273,t_274,t_275,t_276,t_277,t_278,t_279,t_280,t_281,t_282,t_283,t_284,t_285,t_286,t_287,t_288,t_289,t_290,t_291,t_292,t_293,t_294,t_295,t_296,t_297,t_298,t_299
35899,Le

Let's check the shapes of our final Train and Test sets:

In [27]:
#Check X Train
X_train_w2v.shape

(31231, 640)

In [28]:
#Check X Test
X_test_w2v.shape

(7808, 640)

They have 640 columns, which makes sense since each vectorization has 300 dimensions, plus the 40 initial features. They are also a lot smaller than the dimensions we were working on with the Bag-of-Words method! That should make running pipelines a lot faster.

## 1.3 Logistic Regression

We are now ready to run our first logistic regression with the embedded word vectors! First we drop title and description from the X.

In [29]:
#Drop Description and title from the X
X_train_w2v.drop(['title','description'],axis=1,inplace=True)
X_test_w2v.drop(['title','description'],axis=1,inplace=True)

In [30]:
#Checking
X_train_w2v.head()

,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59,t_60,t_61,t_62,t_63,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,t_100,t_101,t_102,t_103,t_104,t_105,t_106,t_107,t_108,t_109,t_110,t_111,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121,t_122,t_123,t_124,t_125,t_126,t_127,t_128,t_129,t_130,t_131,t_132,t_133,t_134,t_135,t_136,t_137,t_138,t_139,t_140,t_141,t_142,t_143,t_144,t_145,t_146,t_147,t_148,t_149,t_150,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160,t_161,t_162,t_163,t_164,t_165,t_166,t_167,t_168,t_169,t_170,t_171,t_172,t_173,t_174,t_175,t_176,t_177,t_178,t_179,t_180,t_181,t_182,t_183,t_184,t_185,t_186,t_187,t_188,t_189,t_190,t_191,t_192,t_193,t_194,t_195,t_196,t_197,t_198,t_199,t_200,t_201,t_202,t_203,t_204,t_205,t_206,t_207,t_208,t_209,t_210,t_211,t_212,t_213,t_214,t_215,t_216,t_217,t_218,t_219,t_220,t_221,t_222,t_223,t_224,t_225,t_226,t_227,t_228,t_229,t_230,t_231,t_232,t_233,t_234,t_235,t_236,t_237,t_238,t_239,t_240,t_241,t_242,t_243,t_244,t_245,t_246,t_247,t_248,t_249,t_250,t_251,t_252,t_253,t_254,t_255,t_256,t_257,t_258,t_259,t_260,t_261,t_262,t_263,t_264,t_265,t_266,t_267,t_268,t_269,t_270,t_271,t_272,t_273,t_274,t_275,t_276,t_277,t_278,t_279,t_280,t_281,t_282,t_283,t_284,t_285,t_286,t_287,t_288,t_289,t_290,t_291,t_292,t_293,t_294,t_295,t_296,t_297,t_298,t_299
12988,479,10,0,1,231,1,0,1

We start with a pipeline on a vanilla Logistic Regression model. We'll use a Standard Scaler here, since it has been performing well before. We will keep C as 1 for now.

In [31]:
#Quick pipeline with vanilla LR

# define steps with the specified objects and parameters
estimators = [
    ('standard_scaling', StandardScaler()),
    ('logreg', LogisticRegression(C=1))
]

# create pipeline object
pipe = Pipeline(estimators)

# fit the pipeline to the untransformed training data
pipe.fit(X_train_w2v, y_train);

# accuracy scores
print(f'train accuracy: {pipe.score(X_train_w2v, y_train)}')
print(f'test accuracy: {pipe.score(X_test_w2v, y_test)}')

train accuracy: 0.6971918926707438
test accuracy: 0.6654713114754098


The test accuracy is 66.5%, better than our best LR model with BoW (66.10%), and we haven't even optimized the parameters yet. Let's do a pipeline to optimize the following parameters:
- C value
- Scaler
- PCA
- Penalty and solver

We'll do a 3-fold cross validation as part of the GridSearch.

In [32]:
#GridSearch to optimize the parameters

# instantiate the estimators

estimators = [
    ('scaling', StandardScaler()),
    ('reduce_dim', PCA()),
    ('model', LogisticRegression())
]

# instantiate pipeline with the specified steps
pipe = Pipeline(estimators)

# define parameter grid
param_grid = [
    
    # L1 regularization and PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [PCA()],
        'reduce_dim__n_components': [0.9],
        'model': [LogisticRegression()],
        'model__penalty': ['l1'],
        'model__solver': ['liblinear'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L1 regularization no PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [None],
        'model': [LogisticRegression()],
        'model__penalty': ['l1'],
        'model__solver': ['liblinear'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L2 regularization and PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [PCA()],
        'reduce_dim__n_components': [0.9],
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L2 regularization and no PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [None],
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
]

# instantiate cross-validated grid search object with the steps and parameter grid
grid = GridSearchCV(
    estimator = pipe,
    param_grid = param_grid,
    cv = 3,
    verbose = 5,
    n_jobs = -1
)

# fit the grid to the unprocessed training data
grid.fit(X_train_w2v, y_train);


Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 3/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=  10.1s
[CV 1/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   9.3s
[CV 1/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   9.5s
[CV 1/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   7.3s
[CV 2/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l1

[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   6.8s
[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   8.4s
[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   9.2s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   9.3s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_

[CV 3/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   9.8s
[CV 3/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   8.6s
[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   8.5s
[CV 2/3] END model=LogisticRegression(), model__C=1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   8.2s
[CV 3/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_compon

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

In [33]:
# check the most optimal model
grid.best_estimator_

Pipeline(steps=[('scaling', StandardScaler()), ('reduce_dim', None),
                ('model',
                 LogisticRegression(C=0.1, penalty='l1', random_state=1,
                                    solver='liblinear'))])

This pipeline ran a lot faster than with the Bag-of-Words dataset! We get the following best parameters: Standard Scaler, no PCA, C=0.1, penalty = l1, solver=liblinear. Let's fit the best model now.

In [34]:
#Scale the train/test with a Standard Scaler

# instantiate the scaler
scaler=StandardScaler()

# fit the scaler to the X_train data
scaler=scaler.fit(X_train_w2v)

# transform both the X_train and X_test data with the fitted scaler
X_train_w2v_ss=scaler.transform(X_train_w2v)
X_test_w2v_ss=scaler.transform(X_test_w2v)

In [35]:
#Instantiate the final LR model
lr=LogisticRegression(C=0.1, penalty='l1', random_state=1, solver='liblinear')

#Fit the model on train data
final_logreg=lr.fit(X_train_w2v_ss, y_train)

#Score the model
print("Train score: ",final_logreg.score(X_train_w2v_ss,y_train))
print("Test score: ",final_logreg.score(X_test_w2v_ss,y_test))

Train score:  0.6968716979923794
Test score:  0.6658555327868853


The accuracy on the test set is a tiny bit better than the one before, 66.6%, but that has not massively improved our model. As with all the other logitic regression models, we can look at the coefficients:

In [36]:
#putting the coef in a dataframe
coef_df=pd.DataFrame(final_logreg.coef_[0],X_train_w2v.columns)
coef_df.sort_values(by=[0],ascending=False).head(10)

,0
pages,0.589845
is_series,0.351629
genre_Childrens,0.185441
book_count_by_author,0.171105
genre_Poetry,0.128627
genre_Classics,0.126451
d_79,0.112343
d_92,0.108255
genre_Other,0.102384
genre_Nonfiction,0.097918


Here again we get the same predictive features as before. But as expected, we see that the text cannot be interpreted in the case of word embedding, as we get a vector dimension and not a word. 

## 1.4 KNN

Let's run a KNN, this was our best performing model earlier with 67.80% accuracy. We'll set a pipeline with a Standard Scaler and K=5 to start.

In [37]:
#Instantiate the parameters

estimators = [
    ('standard_scaling', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# create pipeline object
pipe = Pipeline(estimators)

# fit the pipeline to the untransformed training data
pipe.fit(X_train_w2v, y_train);

# accuracy scores
print(f'Train accuracy: {pipe.score(X_train_w2v, y_train)}')
print(f'Test accuracy: {pipe.score(X_test_w2v, y_test)}')


Train accuracy: 0.7583490762383529
Test accuracy: 0.6075819672131147


The test score is 60.7% and the train score is 75.8%, that looks like overfitting... Let's try to improve that but finding the best K with a Grid Search CV:

In [38]:
#finding the best K with grid search

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, {"n_neighbors": range(1,61,4)})

grid_search.fit(X_train_w2v, y_train)

print("Best parameters: ", grid_search.best_params_)

knn = grid_search.best_estimator_


[CV 2/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   7.1s
[CV 1/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   7.1s
[CV 2/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   8.9s
[CV 1/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   9.0s
[CV 3/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_di

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 3/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   8.0s
[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   8.3s
[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=  10.5s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   8.4s
[CV 2/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_di

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   9.8s
[CV 2/3] END model=LogisticRegression(), model__C=100, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   9.4s
[CV 1/3] END model=LogisticRegression(), model__C=1000, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   8.7s
[CV 3/3] END model=LogisticRegression(), model__C=1000, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=  10.2s
[CV 2/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=RobustScaler(); tot

...

[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   8.6s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   8.1s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   9.9s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=  10.4s
[CV 1/3] END model=LogisticRegression(), model__C=1000, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 3/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=StandardScaler(); total time=   1.5s
[CV 3/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=StandardScaler(); total time=   1.8s
[CV 2/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=StandardScaler(); total time=   2.7s
[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=MinMaxScaler(); total time=   4.0s
[CV 1/3] END model=LogisticRegression(), model__C=1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=MinMaxScaler(); total time=   4.0s
[CV 1/3] END model=LogisticRegression(), model__C=1, model__penalty=l2, model__random_state=1, model

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

Best parameters:  {'n_neighbors': 37}


The cross-validation returns the best K as being 37. We can fit a new model with this hyperparameter. We'll also use a MinMaxScaler to scale the data as this is what performed well earlier on KNN models, as we know that KNN models are highly sensitive to distance.

In [39]:
#Fitting with the best K=37

#Instantiate the parameters

estimators = [
    ('minmax_scaling', MinMaxScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=57))
]

# create pipeline object
pipe = Pipeline(estimators)

# fit the pipeline to the untransformed training data
pipe.fit(X_train_w2v, y_train);

# accuracy scores
print(f'Train accuracy: {pipe.score(X_train_w2v, y_train)}')
print(f'test accuracy: {pipe.score(X_test_w2v, y_test)}')


Train accuracy: 0.676315199641382
test accuracy: 0.6509989754098361


We got a decent accuracy of 65.1%, but not as good as our LR.

We will not run a Decision Tree or Random Forest on this dataset, as it has way too many features to be efficient.

## 1.6 Neural Network

On type of model that we haven't tried before is a neural network. Neural Networks take a lot of fine tuning to get optimized, and we won't do this here as that could be a whole project in itself, but we will run one to get a baseline performance.

In [40]:
import tensorflow as tf
from tensorflow import random as tf_random
from tensorflow import keras
from tensorflow.keras import layers

In [41]:
# Create a new sequential model
model = keras.Sequential()

# Declare the hidden layers (here, 5 hidden layers on 5 node each)
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))

# Declare the output layer (here, 1 output layer)
model.add(layers.Dense(1, activation="sigmoid")) #use sigmoid for binary problems

#Compile the model
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # Metric used to evaluate model
    metrics=[keras.metrics.BinaryAccuracy()]
)

#train the model
history = model.fit(X_train_w2v, y_train, epochs=10, verbose=1)

2022-12-11 20:28:17.739927: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
976/976 [==============================] - 1s 586us/step - loss: 0.6953 - binary_accuracy: 0.5262
Epoch 2/10
976/976 [==============================] - 1s 598us/step - loss: 0.6810 - binary_accuracy: 0.5400
Epoch 3/10
976/976 [==============================] - 1s 597us/step - loss: 0.6761 - binary_accuracy: 0.5553
Epoch 4/10
976/976 [==============================] - 1s 596us/step - loss: 0.6651 - binary_accuracy: 0.5924
Epoch 5/10
976/976 [==============================] - 1s 598us/step - loss: 0.6502 - binary_accuracy: 0.6188
Epoch 6/10
976/976 [==============================] - 1s 601us/step - loss: 0.6353 - binary_accuracy: 0.6414
Epoch 7/10
976/976 [==============================] - 1s 625us/step - loss: 0.6276 - binary_accuracy: 0.6506
Epoch 8/10
976/976 [==============================] - 1s 598us/step - loss: 0.6234 - binary_accuracy: 0.6540
Epoch 9/10
976/976 [==============================] - 1s 592us/step - loss: 0.6220 - binary_accuracy: 0.6558
Epoch 10/10
976/976

In [42]:
model.evaluate(X_test_w2v,y_test, verbose=0)

[0.6191454529762268, 0.6559938788414001]

This looks great, we got an accuracy of 69.2% on the test set! Unfortunately, we don't have the time here or the knowledge required to dig further into neural networks and optimize the performance, but this is definitely something we can explore in our next steps.

# 2. GloVe

GloVe is another type of word embedding developped at Stanford. To work on the same vocabulary as Word2Vec, we have downloaded the Wikipedia 300d vector file on our local drive.

To follow along, you will need to **download the word vector file** from [here](https://nlp.stanford.edu/projects/glove/), save it on your local drive, and change the data path below to match the path of your file.

We have used the code in [this notebook](https://web.stanford.edu/class/cs224n/materials/Gensim%20word%20vector%20visualization.html) to upload the pretrained vectors.

In [43]:
#import the gensim librairies
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [44]:
#Import the pretrained vectors
glove_file = datapath('/Users/lauramercy/Documents/Capstone data/data/glove.6B.300d.txt') #Insert the full file path here
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 300)

Now that we have uploaded the pretrained vectors, we can fit the model and use it to transform the title and description again.

In [45]:
#fit the model
model_glove = KeyedVectors.load_word2vec_format(word2vec_glove_file)

Just like Word2Vec, we can look at similar words in the model. Here, instead of using cosine distance, we can use a built-in fuction 'most_similar' that will return words that are similar to the one selected. We can try with 'book'.

In [46]:
#Check the similarities with 'book'
model_glove.most_similar('book')

[('books', 0.7986249923706055),
 ('author', 0.7123498916625977),
 ('published', 0.6973031163215637),
 ('novel', 0.6966710090637207),
 ('memoir', 0.6465641260147095),
 ('wrote', 0.631791889667511),
 ('biography', 0.6225203275680542),
 ('autobiography', 0.603348970413208),
 ('essay', 0.5995662808418274),
 ('illustrated', 0.5914924144744873)]

Looking at words that are similar to 'book', we get author, published, novel, memoir, essay, etc. This is pretty good!

We'll copy the function we used earlier, but apply it to the GloVe model instead.

In [47]:
#Same function as earlier but with this new model

#Created a function to vectorize the description

def sentence2vec_glove(text):
    """
    Embed a text by averaging the word vectors of the tokenized text. Out-of-vocabulary words are replaced by the zero-vector.
    -----
    
    Input: text (string)
    Output: embedding vector (np.array)
    """
    tokenized_glove = simple_preprocess(text)
    
    word_embeddings_glove = [np.zeros(300)]
    for word in tokenized_glove:
        # if the word is in the model then embed
        if word in model_glove:
            vector_glove = model_glove[word]
        # add zeros for out-of-vocab words
        else:
            vector_glove = np.zeros(300)
            
        word_embeddings_glove.append(vector_glove)
    
    # average the word vectors
    sentence_embedding_glove = np.stack(word_embeddings_glove).mean(axis=0)
    
    return sentence_embedding_glove

Just to be safe, we also reset our X and y, and resplit the dataset into a train and test set. Since we use the same random state, we'll get the same split.

In [48]:
#Assigning X and y
X=books_df.drop('is_popular',axis=1)
y=books_df['is_popular']

In [49]:
# Split the data to train and test set
X_train, X_test, y_train, y_test= train_test_split (X, y, test_size=0.2, stratify=y, random_state=1)

Then we run the same steps as earlier to vectorize Description and Title, and merge the datasets.

## 2.1 Converting description

#### Train set

In [50]:
#Applying sentence2vec_glove to description
X_train_emb_desc = X_train['description'].apply(sentence2vec_glove)

#Putting the results into a numpy array
X_train_emb_desc_np = np.array(X_train_emb_desc.tolist())

#Creating a dataframe
X_train_emb_desc_df=pd.DataFrame(X_train_emb_desc_np, columns=range(0,300,1),index=X_train_emb_desc.index)

#Adding _t in front of the column name
X_train_emb_desc_df=X_train_emb_desc_df.add_prefix("d_")
                            
#Merging with the dataframe created earlier with the description vectors
X_train_glove=pd.merge(X_train,X_train_emb_desc_df,left_index=True,right_index=True)

#checking the new DF
X_train_glove.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
12988,The Radium Girls: The Dark Story of America's ...,\n The incredible true story of the women who...,479,10,0,1,231,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2017,-0.049700,0.083325,0.016158,-0.220341,-0.027792,0.076851,-0.069246,0.060936,0.066995,-1.255382,0.035086,-0.034210,-0.105080,0.053414,0.046389,0.071321,-0.073239,-0.020547,-0.053993,-0.016297,-0.098079,0.163786,0.151742,0.136303,-0.089677,-0.071917,0.095328,-0.019854,-0.087903,0.082448,0.037514,0.136396,-0.231416,0.048824,-0.570680,0.049369,0.012432,0.004282,0.071799,-0.055136,0.101334,-0.161886,-0.017258,0.096203,0.132038,0.052197,0.027095,0.093204,-0.080528,-0.014592,0.074351,-0.063902,0.088133,-0.036808,-0.044786,0.134675,-0.019453,0.113190,0.153340,0.032833,0.020733,-0.020768,0.238762,0.005329,0.012863,-0.211077,0.095189,0.094282,0.000602,0.021878,0.030833,0.035066,0.009068,-0.026507,-0.037278,0.014844,0.015984,-0.057215,-0.131420,-0.007179,-0.143655,-0.102782,0.086408,0.130026,0.018331,0.070549,-0.013205,0.100216,0.075268,0.098629,-0.109074,0.026976,-0.094182,0.005409,-0.043862,0.040977,-0.168441,-0.019979,0.093494,-0.256561,0.097310,0.102554,-0.046853,0.006065,-0.017551,-0.041461,0.119842,0.048027,-0.098658,0.115060,0.062227,-0.087360,-0.072222,-0.129240,0.106553,0.130136,0.022763,0.099613,0.005095,-0.209885,-0.082909,-0.126533,0.010536,0.167114,-0.048454,0.006244,-0.029774,0.136945,0.047031,0.067032,0.059227,0.123128,0.117342,-0.010693,-0.118890,0.013808,-0.040602,0.010288,-0.049039,0.145770,-0.009172,0.024881,-0.107271,0.137433,-0.138145,-0.075241,0.082810,0.042076,-0.036371,0.027776,0.234453,-0.097587,0.007690,0.030985,0.296157,0.014063,-0.118672,-0.010321,0.007180,-0.075665,0

#### Test set

In [51]:
#Applying sentence2vec_glove to description
X_test_emb_desc = X_test['description'].apply(sentence2vec_glove)

#Putting the results into a numpy array
X_test_emb_desc_np = np.array(X_test_emb_desc.tolist())

#Creating a dataframe
X_test_emb_desc_df=pd.DataFrame(X_test_emb_desc_np, columns=range(0,300,1),index=X_test_emb_desc.index)

#Adding _t in front of the column name
X_test_emb_desc_df=X_test_emb_desc_df.add_prefix("d_")
                            
#Merging with the dataframe created earlier with the description vectors
X_test_glove=pd.merge(X_test,X_test_emb_desc_df,left_index=True,right_index=True)

#checking the new DF
X_test_glove.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299
35899,Legends over Generations,"Since the beginning of human settlement, a lot...",228,3,0,1,188,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2018,-0.102875,0.089236,-0.026890,-0.155465,0.001404,0.076663,-0.154656,0.056705,0.066666,-1.498400,0.102582,-0.029151,-0.085115,0.069946,0.038749,0.009039,0.003877,-0.019806,-0.086954,0.033226,0.015953,0.231343,0.151944,0.140647,-0.088327,-0.011435,0.052774,-0.005565,-0.050244,0.071794,0.099130,0.210195,-0.257993,0.051699,-0.702324,0.081910,0.127676,0.068014,0.001568,-0.090670,0.092901,-0.180135,-0.058734,0.036132,0.103614,0.137074,0.060461,0.204052,-0.090153,0.064338,0.125689,-0.129210,0.054293,-0.100430,-0.069140,0.213045,-0.025285,0.142858,0.091587,0.036555,0.112765,0.003918,0.199931,0.001710,-0.013770,-0.167742,0.102526,0.177255,-0.058339,0.048269,-0.021550,-0.014148,-0.004467,0.064585,-0.012515,-0.016068,0.063951,0.090911,-0.099091,-0.037753,-0.114387,-0.096865,0.099102,0.046925,0.027709,0.122980,0.006679,0.105846,0.061905,0.119128,-0.178202,0.113400,-0.132713,-0.001492,-0.000950,-0.019655,-0.163548,-0.001359,0.071513,-0.278885,0.069385,0.066117,-0.018531,-0.025388,-0.059918,0.049284,0.105721,0.053048,-0.083057,0.083497,0.052707,-0.185514,-0.096223,-0.167484,0.094115,0.077486,-0.009159,0.058533,-0.008015,-0.182518,-0.043227,-0.075181,0.016028,0.155671,-0.043909,-0.001323,-0.018618,0.146012,0.028823,-0.004600,0.005607,0.029942,0.133666,-0.058616,-0.139226,-0.061341,-0.016329,-0.067409,-0.077908,0.199984,-0.083279,0.007400,0.007138,0.140331,-0.103234,-0.018475,0.034671,0.026333,-0.098677,0.067002,0.154107,-0.057973,0.057788,0.025068,0.218037,0.022555,-0.169430,0.045945,0.119284,-0.164776,0.070701,-0.085755,0

## 2.2 Converting title

#### Train set

In [52]:
#Applying sentence2vec_glove to titleription
X_train_emb_title = X_train['title'].apply(sentence2vec_glove)

#Putting the results into a numpy array
X_train_emb_title_np = np.array(X_train_emb_title.tolist())

#Creating a dataframe
X_train_emb_title_df=pd.DataFrame(X_train_emb_title_np, columns=range(0,300,1),index=X_train_emb_title.index)

#Adding _t in front of the column name
X_train_emb_title_df=X_train_emb_title_df.add_prefix("t_")
                            
#Merging with the dataframe created earlier with the titleription vectors
X_train_glove=pd.merge(X_train,X_train_emb_title_df,left_index=True,right_index=True)

#checking the new DF
X_train_glove.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59,t_60,t_61,t_62,t_63,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,t_100,t_101,t_102,t_103,t_104,t_105,t_106,t_107,t_108,t_109,t_110,t_111,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121,t_122,t_123,t_124,t_125,t_126,t_127,t_128,t_129,t_130,t_131,t_132,t_133,t_134,t_135,t_136,t_137,t_138,t_139,t_140,t_141,t_142,t_143,t_144,t_145,t_146,t_147,t_148,t_149,t_150,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160,t_161,t_162,t_163,t_164,t_165,t_166,t_167,t_168,t_169,t_170,t_171,t_172,t_173,t_174,t_175,t_176,t_177,t_178,t_179,t_180,t_181,t_182,t_183,t_184,t_185,t_186,t_187,t_188,t_189,t_190,t_191,t_192,t_193,t_194,t_195,t_196,t_197,t_198,t_199,t_200,t_201,t_202,t_203,t_204,t_205,t_206,t_207,t_208,t_209,t_210,t_211,t_212,t_213,t_214,t_215,t_216,t_217,t_218,t_219,t_220,t_221,t_222,t_223,t_224,t_225,t_226,t_227,t_228,t_229,t_230,t_231,t_232,t_233,t_234,t_235,t_236,t_237,t_238,t_239,t_240,t_241,t_242,t_243,t_244,t_245,t_246,t_247,t_248,t_249,t_250,t_251,t_252,t_253,t_254,t_255,t_256,t_257,t_258,t_259,t_260,t_261,t_262,t_263,t_264,t_265,t_266,t_267,t_268,t_269,t_270,t_271,t_272,t_273,t_274,t_275,t_276,t_277,t_278,t_279,t_280,t_281,t_282,t_283,t_284,t_285,t_286,t_287,t_288,t_289,t_290,t_291,t_292,t_293,t_294,t_295,t_296,t_297,t_298,t_299
12988,The Radium Girls: The Dark Story of America's ...,\n The incredible true story of the women who...,479,10,0,1,231,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2017,-0.040325,0.039269,0.110273,-0.242625,-0.097445,0.239915,-0.238254,0.198467,0.079286,-1.006462,0.116365,-0.108609,-0.014475,0.133801,-0.024387,-0.052952,0.017288,-0.149766,-0.077811,-0.118692,-0.283691,0.237369,0.119788,0.293580,-0.051854,-0.098498,0.268055,-0.073370,-0.160810,0.071449,0.060318,0.136532,-0.296974,-0.025328,-0.457694,0.249623,-0.025999,-0.041804,0.217465,-0.065923,0.302489,-0.359687,-0.112149,0.160154,0.200807,0.027516,-0.019973,0.115140,-0.163143,-0.171308,-0.041803,0.049289,0.159892,-0.040448,-0.054972,0.098631,-0.093083,0.122405,0.299814,-0.118874,-0.030457,-0.028952,0.321911,-0.063036,0.035232,-0.193095,0.071876,0.114354,-0.013501,-0.044670,0.092103,-0.030426,-0.095731,-0.117699,-0.108828,-0.029723,0.014618,-0.242688,-0.155454,-0.145745,-0.116610,-0.023948,0.159131,0.137573,-0.065761,0.030951,-0.001999,0.094645,0.108466,0.082391,-0.162578,-0.143283,-0.021943,0.141303,-0.048304,0.066475,-0.150877,0.010011,0.158731,-0.325378,0.141554,0.108565,-0.127558,0.133115,0.039845,-0.191230,0.164935,0.030432,-0.046595,0.242309,0.016994,-0.045160,0.134950,-0.129619,0.153922,0.201262,0.045305,0.164423,-0.115519,-0.260027,-0.146455,-0.140027,0.167090,0.175594,-0.076646,0.075864,-0.050365,0.144281,-0.078753,0.069636,0.044694,0.079379,0.069079,0.043968,-0.110708,0.117893,-0.206871,0.185858,-0.232783,0.081441,-0.008008,-0.004496,-0.231931,0.060994,-0.011402,-0.250422,0.133178,0.201584,0.080421,0.067790,0.276986,-0.158459,0.089514,0.069317,0.349937,-0.085296,-0.076534,0.007317,0.04163

#### Test Set

In [53]:
#Applying sentence2vec_glove to title
X_test_emb_title = X_test['title'].apply(sentence2vec_glove)

#Putting the results into a numpy array
X_test_emb_title_np = np.array(X_test_emb_title.tolist())

#Creating a dataframe
X_test_emb_title_df=pd.DataFrame(X_test_emb_title_np, columns=range(0,300,1),index=X_test_emb_title.index)

#Adding _t in front of the column name
X_test_emb_title_df=X_test_emb_title_df.add_prefix("t_")
                            
#Merging with the dataframe created earlier with the title vectors
X_test_glove=pd.merge(X_test,X_test_emb_title_df,left_index=True,right_index=True)

#checking the new DF
X_test_glove.head()

,title,description,pages,word_count_title,is_series,book_count_by_author,word_count_description,genre_Adult,genre_Adventure,genre_Biography,genre_Childrens,genre_Classics,genre_Contemporary,genre_Fantasy,genre_Fiction,genre_Historical,genre_Historical Fiction,genre_History,genre_Horror,genre_Islam,genre_Literature,genre_Magic,genre_Mystery,genre_Nonfiction,genre_Novels,genre_Paranormal,genre_Poetry,genre_Romance,genre_Science Fiction,genre_Thriller,genre_Urban,genre_Young Adult,genre_Other,format_Audiobook,format_Hardcover,format_Mass Market Paperback,format_Paperback,format_ebook,has_awards,publish_year,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,t_11,t_12,t_13,t_14,t_15,t_16,t_17,t_18,t_19,t_20,t_21,t_22,t_23,t_24,t_25,t_26,t_27,t_28,t_29,t_30,t_31,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,t_41,t_42,t_43,t_44,t_45,t_46,t_47,t_48,t_49,t_50,t_51,t_52,t_53,t_54,t_55,t_56,t_57,t_58,t_59,t_60,t_61,t_62,t_63,t_64,t_65,t_66,t_67,t_68,t_69,t_70,t_71,t_72,t_73,t_74,t_75,t_76,t_77,t_78,t_79,t_80,t_81,t_82,t_83,t_84,t_85,t_86,t_87,t_88,t_89,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,t_100,t_101,t_102,t_103,t_104,t_105,t_106,t_107,t_108,t_109,t_110,t_111,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121,t_122,t_123,t_124,t_125,t_126,t_127,t_128,t_129,t_130,t_131,t_132,t_133,t_134,t_135,t_136,t_137,t_138,t_139,t_140,t_141,t_142,t_143,t_144,t_145,t_146,t_147,t_148,t_149,t_150,t_151,t_152,t_153,t_154,t_155,t_156,t_157,t_158,t_159,t_160,t_161,t_162,t_163,t_164,t_165,t_166,t_167,t_168,t_169,t_170,t_171,t_172,t_173,t_174,t_175,t_176,t_177,t_178,t_179,t_180,t_181,t_182,t_183,t_184,t_185,t_186,t_187,t_188,t_189,t_190,t_191,t_192,t_193,t_194,t_195,t_196,t_197,t_198,t_199,t_200,t_201,t_202,t_203,t_204,t_205,t_206,t_207,t_208,t_209,t_210,t_211,t_212,t_213,t_214,t_215,t_216,t_217,t_218,t_219,t_220,t_221,t_222,t_223,t_224,t_225,t_226,t_227,t_228,t_229,t_230,t_231,t_232,t_233,t_234,t_235,t_236,t_237,t_238,t_239,t_240,t_241,t_242,t_243,t_244,t_245,t_246,t_247,t_248,t_249,t_250,t_251,t_252,t_253,t_254,t_255,t_256,t_257,t_258,t_259,t_260,t_261,t_262,t_263,t_264,t_265,t_266,t_267,t_268,t_269,t_270,t_271,t_272,t_273,t_274,t_275,t_276,t_277,t_278,t_279,t_280,t_281,t_282,t_283,t_284,t_285,t_286,t_287,t_288,t_289,t_290,t_291,t_292,t_293,t_294,t_295,t_296,t_297,t_298,t_299
35899,Legends over Generations,"Since the beginning of human settlement, a lot...",228,3,0,1,188,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2018,-0.140514,-0.010017,0.231927,0.036863,-0.000298,0.038727,-0.174860,0.080911,0.067532,-0.411227,0.171915,-0.023943,-0.277470,-0.121580,0.038023,-0.063468,0.033513,0.066922,0.239022,0.321050,-0.011922,0.031809,-0.117634,0.126498,0.147035,-0.067528,-0.087660,0.176386,-0.085335,0.088180,0.159968,0.144162,-0.284173,-0.040763,-0.374880,-0.151143,0.078242,-0.023011,0.165672,0.228643,0.240792,-0.394255,-0.204255,0.046422,0.164225,0.090433,0.021413,0.020640,-0.132298,0.167980,-0.123390,-0.200937,-0.151825,-0.142227,-0.045961,0.109312,0.113473,0.217650,0.002064,0.137892,0.011528,-0.170323,0.225530,-0.043438,0.262032,-0.178418,0.044621,0.343335,-0.175255,0.023800,-0.024453,-0.030890,-0.088509,0.014348,-0.073667,-0.082733,-0.068071,-0.106348,-0.099175,-0.214330,-0.112072,0.100405,0.184908,0.028122,-0.163883,0.049520,0.177590,0.147381,0.204172,0.064085,0.033159,-0.114357,-0.060177,0.328130,-0.139006,-0.128720,0.111072,0.006830,0.127878,-0.201871,0.240712,0.202343,0.191757,-0.290085,0.001375,0.060997,0.181908,-0.120153,-0.229030,0.084228,0.132026,-0.200122,-0.021563,-0.148131,-0.110035,0.088008,0.042234,0.107500,0.042666,-0.262870,0.093239,0.067703,-0.297905,0.203525,-0.119158,0.268803,-0.281312,0.306670,0.022285,0.087012,-0.169525,0.069552,0.188850,0.142382,0.099227,-0.238440,0.043117,0.151680,-0.083404,0.112108,0.089456,-0.346975,0.054063,0.173775,0.077060,0.186640,-0.255760,-0.034583,-0.012965,0.057452,0.013054,0.139959,-0.138854,0.078090,0.147237,0.042787,-0.162635,0.095228,0.007967,-0.114768,0.042817,0.005600,-0.088

For the sake of comparison, we could also calculate the cosine distance between the description in row 1 and 2, and between row 1 and 3, and see if we get similar results as with Word2Vec:

In [54]:
#Distance between the description of row 1 and row 2
cosine_distance(X_train_emb_desc_df.iloc[0], X_train_emb_desc_df.iloc[1])

0.0681514372085904

In [55]:
#Distance between the description of row 1 and row 3
cosine_distance(X_train_emb_desc_df.iloc[0], X_train_emb_desc_df.iloc[2])

0.1590452130253991

## 2.3 Logistic Regression

As before, we drop the description and title, and run a vanilla Logistic Regression.

In [56]:
#Drop Description and title from the X
X_train_glove.drop(['title','description'],axis=1,inplace=True)
X_test_glove.drop(['title','description'],axis=1,inplace=True)

We set up a pipeline with a Standard Scaler and C=1:

In [57]:
#Quick pipeline with vanilla LR

# define steps with the specified objects and parameters
estimators = [
    ('standard_scaling', StandardScaler()),
    ('logreg', LogisticRegression(C=1))
]

# create pipeline object
pipe = Pipeline(estimators)

# fit the pipeline to the untransformed training data
pipe.fit(X_train_glove, y_train);

# accuracy scores
print(f'Train accuracy: {pipe.score(X_train_glove, y_train)}')
print(f'Test accuracy: {pipe.score(X_test_glove, y_test)}')

Train accuracy: 0.6727610387115366
Test accuracy: 0.6445952868852459


We get an accuracy of 64.4%, not as good as with Word2Vec which has 66.5%. Again, we'll do a Grid Search to optimize the parameters:

In [58]:
#GridSearch to optimize the parameters

# instantiate the estimators

estimators = [
    ('scaling', StandardScaler()),
    ('reduce_dim', PCA()),
    ('model', LogisticRegression())
]

# instantiate pipeline with the specified steps
pipe = Pipeline(estimators)

# define parameter grid
param_grid = [
    
    # L1 regularization and PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [PCA()],
        'reduce_dim__n_components': [0.9],
        'model': [LogisticRegression()],
        'model__penalty': ['l1'],
        'model__solver': ['liblinear'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L1 regularization no PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [None],
        'model': [LogisticRegression()],
        'model__penalty': ['l1'],
        'model__solver': ['liblinear'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L2 regularization and PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [PCA()],
        'reduce_dim__n_components': [0.9],
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
    
    # L2 regularization and no PCA
    {
        'scaling': [MinMaxScaler(), StandardScaler(), RobustScaler()],
        'reduce_dim': [None],
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs'],
        'model__C': [0.0001,0.001,0.01,0.1,1,10,100,1000],
        'model__random_state': [1]
    },
]

# instantiate cross-validated grid search object with the steps and parameter grid
grid = GridSearchCV(
    estimator = pipe,
    param_grid = param_grid,
    cv = 3,
    verbose = 5,
    n_jobs = -1
)

# fit the grid to the unprocessed training data
grid.fit(X_train_glove, y_train);


Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   2.6s
[CV 3/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.7s
[CV 2/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.6s
[CV 2/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   2.8s
[CV 1/3] END model=LogisticRegression(), model__C=0.001, model__penalty=l

[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   2.8s
[CV 2/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.3s
[CV 2/3] END model=LogisticRegression(), model__C=1000, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   2.4s
[CV 1/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total time=   0.6s
[CV 3/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total tim

[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.5s
[CV 1/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   2.4s
[CV 3/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.3s
[CV 3/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   2.0s
[CV 1/3] END model=LogisticRegression(), model__C=1000, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=PCA(), reduce

In [59]:
# check the most optimal model
grid.best_estimator_

Pipeline(steps=[('scaling', StandardScaler()), ('reduce_dim', None),
                ('model',
                 LogisticRegression(C=0.01, penalty='l1', random_state=1,
                                    solver='liblinear'))])

The best model is with a StandardScaler, C=0.01, penalty=l1 and solver=linlinear.

In [60]:
#Fitting the best model

# define steps with the specified objects and parameters
estimators = [
    ('standard_scaling', StandardScaler()),
    ('logreg', LogisticRegression(C=0.01, penalty='l1', random_state=1, solver='liblinear'))
]

# create pipeline object
pipe = Pipeline(estimators)

# fit the pipeline to the untransformed training data
pipe.fit(X_train_glove, y_train);

# accuracy scores
print(f'train accuracy: {pipe.score(X_train_glove, y_train)}')
print(f'test accuracy: {pipe.score(X_test_glove, y_test)}')

train accuracy: 0.6694630335243829
test accuracy: 0.6457479508196722


The score is pretty close to was we got before, 64.6% accuracy.

## 2.3 Neural Network

Since our Neural Network had so much success last time, let's try it again on the GloVe vectors. We use the same parameters as before.

In [61]:
# Create a new sequential model
model = keras.Sequential()

# Declare the hidden layers (here, 5 hidden layers on 5 node each)
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))
model.add(layers.Dense(5, activation="relu"))

# Declare the output layer (here, 1 output layer)
model.add(layers.Dense(1, activation="sigmoid")) #use sigmoid for binary problems

#Compile the model
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # Metric used to evaluate model
    metrics=[keras.metrics.BinaryAccuracy()]
)

#train the model
history = model.fit(X_train_glove, y_train, epochs=10, verbose=1)

Epoch 1/10
976/976 [==============================] - 1s 594us/step - loss: 0.6970 - binary_accuracy: 0.5169
Epoch 2/10
976/976 [==============================] - 1s 588us/step - loss: 0.6927 - binary_accuracy: 0.5156
Epoch 3/10
976/976 [==============================] - 1s 580us/step - loss: 0.6927 - binary_accuracy: 0.5156
Epoch 4/10
976/976 [==============================] - 1s 590us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 5/10
976/976 [==============================] - 1s 577us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 6/10
976/976 [==============================] - 1s 582us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 7/10
976/976 [==============================] - 1s 589us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 8/10
976/976 [==============================] - 1s 578us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 9/10
976/976 [==============================] - 1s 591us/step - loss: 0.6927 - binary_accuracy: 0.5157
Epoch 10/10
976/976

In [62]:
model.evaluate(X_test_glove,y_test, verbose=0)

[0.6926602125167847, 0.515625]

[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=   2.4s
[CV 2/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=  13.3s
[CV 1/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=StandardScaler(); total time=  13.7s
[CV 3/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=StandardScaler(); total time=  13.1s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total time= 3.2min
[CV 2/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l2, model__random_s

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 2/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   3.1s
[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   3.4s
[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.8s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=MinMaxScaler(); total time=   2.8s
[CV 1/3] END model=LogisticRegression(), model__C=10, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, 

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 3/3] END model=LogisticRegression(), model__C=10, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.9s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.7s
[CV 3/3] END model=LogisticRegression(), model__C=100, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=RobustScaler(); total time=   3.4s
[CV 2/3] END model=LogisticRegression(), model__C=1000, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=PCA(), reduce_dim__n_components=0.9, scaling=StandardScaler(); total time=   2.7s
[CV 1/3] END model=LogisticRegression(), model__C=0.0001, model__penalty=l2, model__random_state=1, model__solver=lbfgs, reduce_dim=None, scaling=MinMaxScaler

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 2/3] END model=LogisticRegression(), model__C=0.01, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total time=   0.6s
[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=StandardScaler(); total time=   2.4s
[CV 2/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total time=  30.9s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=  18.9s
[CV 1/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=StandardScaler(); total time=  13.8s
[CV 3/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_

/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/lauramercy/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=MinMaxScaler(); total time=   3.4s
[CV 1/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=   8.9s
[CV 3/3] END model=LogisticRegression(), model__C=1, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=   8.5s
[CV 2/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=StandardScaler(); total time=  20.8s
[CV 3/3] END model=LogisticRegression(), model__C=10, model__penalty=l1, model__random_state=1, model__solver=liblinear, reduce_dim=None, scaling=RobustScaler(); total time=  12.3s
[CV 2/3] END model=LogisticRegression(), model__C=100, model__penalty=l1, model__random_state=

We get a similar score, 69.2% accuracy on the test score.

# 3. Conclusion on Word Embeddings

As we have seen, word embeddings are quite powerful at identifying the meaning of a word, and how it relates with other similar words. It is also useful to reduce dimensionality compared to Bag-of-Words or TF-IDF. However, it reduces interpretability as we are dealing with dimensional vectors and not tokens. Neural Networks seem to be promising to improve the model's performance, but require a lot of fine tuning which we decided not to do here.

# 4. Project Conclusion

Through this project we have:
- Cleaned up a pretty complex dataset, with a lot of different formats and inputs
- Collected some data insights
- Ran machine learning models on non-text features
- Performed NLP on title and description, and ran machine learning models with word vectors

We can recap our best model scores here:

Model | Train | Test
:------|:-----------|:------
**Non-text features only**
LR, Standard Scaler, C=0.001 | 65.7% | 63.8%
KNN, Robust Scaler, K=37 | 70.6% | 67.8%
Decision Tree, depth=6 |66.1% | 64.5%
Random Forest, n=60, depth=5 | 67.0% | 65.5%
**BoW**
LR, C=0.001, Description only |66.5% |62.4%
LR, C=0.001, Title only | 51.6% |51.6%
LR, Robust Scaler, C=0.001 | 69.1% |66.1%
**TF-IDF**
LR, C=0.001, Description only |69.7% |64.1%
LR, C=0.001, Title only | 60.50% | 56%
**Word2Vec**
LR, Standard Scaler, C=0.1 | 69.7% | 66.6%
KNN, Robust Scaler, K=37 | 67.6% | 65.1%
Neural Network | |69.3%
**GloVe**
LR, Standard Scaler, C=1 | 67.3% | 64.4%
LR, Standard Scaler, C=0.01 | 66.9% | 64.8%
Neural Network | |69.3%

If we take a step back and remind ourselves what the prediction would be if we left it at chance only, we would get 52%. Indeed, in our dataset, 52% of the books are popular and 48% are not. Our predictions here vary between 62% to 69%. While this is better than chance, this is lower than what we expected. 

Let's bring it back to our business question - can we predict the popularity of a book on the Goodreads' list, and what features are predictive? A few findings were pretty consistent through both the EDA and the modeling as they came up in the Logistic Regression highest coefficients, in the Random Forest's most important features, and in the Bag-of-Words most predictive coefficients on the description:
- being part of a series: volume and series were in the highest coefficients for predictive words
- the length of a book (average number of pages is 320): longer books tend to be more popular
- the number of books that the author has listed on the Goodreads' list: this seems pretty intuitive, it goes with the popularity of the author

Books that address battles and wars also tend to be popular. We don't want to put too much weight on prective power of genres, as we have seen that they are not equally distributed through the dataset.

In our next steps, we would like to explore the following:

- Deep dive into neural networks, as they seem to have quite a better accuracy than the other models on this dataset
- Add new features: the Goodreads' list can be quite subjective to judge the popularity of a book, and adding more objective metrics like the sales volume could help our preditions
- Perform more analysis and modeling on the description - we haven't looked at similarities at all, and we suppose that books with similar themes might also have similar popularity. 
